# [CSC8101] Engineering for AI - 2024 Spark Coursework

## Coursework overview

### Inputs

- **NYC Taxi Trips dataset** - list of recorded taxi trips, each with several characteristics, namely: distance, number of passengers, origin zone, destination zone and trip cost (total amount charged to customer).
- **NYC Zones dataset** - list of zones wherein trips can originate/terminate.

### Tasks

1. Data cleaning
  1. Remove "0 distance" and 'no passengers' records.
  2. Remove outlier records. 
2. Add new columns
  1. Join with zones dataset
  2. Compute the unit profitability of each trip
3. Zone summarisation and ranking
  1. Summarise trip data per zone
  2. Obtain the top 10 ranks according to:
    1. The total trip volume
    2. Their average profitabilitiy
    3. The total passenger volume
4. Record the total and task-specific execution times for each dataset size and format.

### How to

###### Code structure and implementation

- You must implement your solution to each task in the provided function code skeleton.
- The task-specific functions are combined together to form the full pipeline code, executed last (do not modify this code).
- Before implementing the specified function skeleton, you should develop and test your solution on separate code cells (create and destroy cells as needed).

###### Development

- Develop an initial working solution for the 'S' dataset and only then optimise it for larger dataset sizes.
- To perform vectorised operations on a DataFrame:
  - use the API docs to look for existing vectorised functions in: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html
  - actions to get around the lazy execution of spark: 
  https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions
  - if a customised function is required (e.g. to add a new column based on a linear combination of other columns), implement your own User Defined Function (UDF). See:  https://spark.apache.org/docs/latest/sql-ref-functions-udf-scalar.html
- Use only the `pyspark.sql` API - documentation link below - (note that searching through the docs returns results from the `pyspark.sql` API together with the `pyspark.pandas` API):
  - https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.sql.html
- Periodically download your notebook to your computer as backup and safety measure against accidental file deletion.
 
###### Execution time measurement

- Execution time is calculated and returned by the Spark Engine and shown in the output region of the cell.
- To measure the execution time of a task you must perform a `collect` or similar operation (e.g. `take`) on the returned DataFrame.

## Task 0 - Read data

The code below is ready to run. **Do not modify this code**. It does the following:

- Reads the 'zones' dataset into variable 'zone_names'
- Defines the `init_trips` function that allows you to read the 'trips' dataset (from the DBFS FileStore) given the dataset size ('S' to 'XXL') and format ('parquet' or 'delta') as function arguments
- Defines the `pipeline` function, called in Task 4 to measure the execution time of the entire data processing pipeline
- Shows you how to call the `init_trips` function and display dataset characteristics (number of rows, schema)

In [0]:
## global imports
import pyspark.sql as ps
import pyspark.sql.functions as pf
import pandas as pd

# Load zone names dataset - (much faster to read small file from git than dbfs)
zones_file_url = 'https://raw.githubusercontent.com/mutazb999/CSC8101-lab-and-coursework/main/02-assignment-spark/taxi_zone_names.csv'
zone_names = spark.createDataFrame(pd.read_csv(zones_file_url))

# Function to load trips dataset by selected dataset size
def init_trips(size = 'S', data_format = "parquet", taxi_folder = "/FileStore/tables/taxi"):     
    
    files = {
        'S'  : ['2021_07'],
        'M'  : ['2021'],
        'L'  : ['2020_21'],
        'XL' : ['1_6_2019', '7_12_2019'],
        'XXL': ['1_6_2019', '7_12_2019', '2020_21']
    }
    
    # validate input dataset size
    if size not in files.keys():
        print("Invalid input dataset size. Must be one of {}".format(list(files.keys())))
        return None               
    
    if data_format == "parquet":
        filenames = list(map(lambda s: f'{taxi_folder}/parquet/tripdata_{s}.parquet', files[size]))
        trips_df = spark.read.parquet(filenames[0])
        
        for name in filenames[1:]:
            trips_df = trips_df.union(spark.read.parquet(name))
            
    elif data_format == "delta":
        filenames = f"{taxi_folder}/delta/taxi-{size}-delta/"
        trips_df = spark.read.format("delta").load(filenames)
    
    else:
        print("Invalid data format. Must be one of {}".format(['parquet', 'delta']))
        return None
        
    print(
    """
    Trips dataset loaded!
    ---
      Size: {s}
      Format: {f}
      Tables loaded: {ds}
      Number of trips (dataset rows): {tc:,}
    """.format(s = size, f = data_format, ds = filenames, tc = trips_df.count()))
    
    return trips_df

# helper function to print dataset row count
def print_count(df):
    print("Row count: {t:,}".format(t = df.count()))

def pipeline(trips_df, with_task_12 = False, zones_df = zone_names):
    # Do not edit
    #---

    ## Task 1.1
    _trips_11 = t11_remove_zeros(trips_df)

    ## Task 1.2
    if with_task_12:
        _trips_12 = t12_remove_outliers(_trips_11)
    else:
        _trips_12 = _trips_11

    ## Task 2.1
    _trips_21 = t21_join_zones(_trips_12, zones_df = zone_names)

    ## Task 2.2
    _trips_22 = t22_calc_profit(_trips_21)

    ## Task 3.1
    _graph = t31_summarise_trips(_trips_22)

    ## Task 3.2
    _zones = t32_summarise_zones_pairs(_graph)

    _top10_trips     = t32_top10_trips(_zones)
    _top10_profit    = t32_top10_profit(_zones)
    _top10_passenger = t32_top10_passenger(_zones)
    
    return([_top10_trips, _top10_profit, _top10_passenger])

In [0]:
# CHANGE the value of argument 'size' to record the pipeline execution times for increasing dataset sizes
SIZE = 'S'
DATA_FORMAT = 'parquet'

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)

# uncomment line only for small datasets
# trips.take(1)


    Trips dataset loaded!
    ---
      Size: S
      Format: parquet
      Tables loaded: ['/FileStore/tables/taxi/parquet/tripdata_2021_07.parquet']
      Number of trips (dataset rows): 2,898,033
    


In [0]:
print_count(trips)

Row count: 2,898,033


In [0]:
# dataset schemas
trips.printSchema()

root
 |-- index: long (nullable = true)
 |-- VendorID: double (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- cab_type: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |--

In [0]:
display(trips[['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']].take(5))

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
90,68,0.8,1.0,8.8
113,90,0.9,1.0,8.8
88,232,2.8,1.0,13.8
79,249,1.4,1.0,12.3
142,238,2.0,0.0,12.3


In [0]:
zone_names.printSchema()

root
 |-- LocationID: long (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [0]:
display(zone_names.take(5))

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone


## Task 1 - Filter rows

**Input:** trips dataset

### Task 1.1 - Remove "0 distance" and 'no passengers' records

Remove dataset rows that represent invalid trips:

- Trips where `trip_distance == 0` (no distance travelled)
- Trips where `passenger_count == 0` and `total_amount == 0` (we want to retain records where `total_amount` > 0 - these may be significant as the taxi may have carried some parcel, for example)

Altogether, a record is removed if it satisfies the following conditions:

`trip_distance == 0` or `(passenger_count == 0` and `total_amount == 0)`.

**Recommended:** Select only the relevant dataset columns for this and subsequent tasks: `['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')]`

### Task 1.2 - Remove outliers using the modified z-score

Despite having removed spurious "zero passengers" trips in task 1.1, columns `total_amount` and `trip_distance` contain additional outlier values that must be identified and removed.

To identify and remove outliers, you will use the modified [z-score](https://en.wikipedia.org/wiki/Standard_score) method.
The modified z-score uses the median and [Median Absolute Deviation](https://en.wikipedia.org/wiki/Median_absolute_deviation) (MAD), instead of the mean and standard deviation, to determine how far an observation (indexed by i) is from the mean:

$$z_i = \frac{x_i - \mathit{median}(\mathbf{x})}{\mathbf{MAD}},$$

where x represents the input vector, xi is an element of x and zi is its corresponding z-score. In turn, the MAD formula is:

$$\mathbf{MAD} = 1.483 * \mathit{median}(\big\lvert x_i - \mathit{median}(\mathbf{x})\big\rvert).$$

Observations with **high** (absolute) z-score are considered outlier observations. A score is considered **high** if its __absolute z-score__ is larger than a threshold T = 3.5:

$$\big\lvert z_i \big\rvert > 3.5.$$

where T represents the number of unit standard deviations beyond which a score is considered an outlier ([wiki](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule)).

This process is repeated twice, once for each of the columns `total_amount` and `trip_distance` (in any order).

**Important:** Use the surrogate function [`percentile_approx`](https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.functions.percentile_approx.html?highlight=percentile#pyspark.sql.functions.percentile_approx) to estimate the median (calculating the median values for a column is expensive as it cannot be parallelised efficiently).

In [0]:
# Your solution implementation to task 1.1 goes HERE
def t11_remove_zeros(df):
    # input: trips dataset
    df_red = df.select('PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')
    filtered_df = df_red.filter(~((df_red['trip_distance'] == 0) | ((df_red['passenger_count'] == 0) & (df_red['total_amount'] == 0))))
    return filtered_df

In [0]:
# execute task 1.1
trips_11 = t11_remove_zeros(trips)

print_count(trips_11)

## uncomment only for smaller datasets
display(trips_11.take(10))

Row count: 2,858,164


PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
90,68,0.8,1.0,8.8
113,90,0.9,1.0,8.8
88,232,2.8,1.0,13.8
79,249,1.4,1.0,12.3
142,238,2.0,0.0,12.3
114,90,1.6,1.0,12.8
90,144,1.8,1.0,13.3
114,48,2.0,1.0,14.75
48,152,5.7,1.0,22.3
234,148,1.8,1.0,14.75


In [0]:
# Your solution implementation to task 1.2 goes HERE
from pyspark.sql.functions import abs

#Method to calculate the median and MAD value of a given column of a dataframe to be used in the function t12_remove_outliers
def calculate_median_mad_value(df, column_name):
    median_query = 'percentile_approx({}, 0.5)'.format(column_name)
    median = df.selectExpr(median_query).collect()[0][0]

    mad_query = '1.483 * percentile_approx(ABS({} - {}), 0.5)'.format(column_name, median)
    mad_value = df.selectExpr(mad_query).collect()[0][0]

    return median, mad_value

def t12_remove_outliers(df):
    # trip_distance
    median, mad_value = calculate_median_mad_value(df, 'trip_distance')
    zi_trip_distance = (df['trip_distance'] - median) / mad_value
    filtered_df = df.filter(abs(zi_trip_distance) <= 3.5)

    # total_amount 
    median_2, mad_value_2 = calculate_median_mad_value(filtered_df, 'total_amount')
    zi_total_amount = (filtered_df['total_amount'] - median_2) / mad_value_2
    filtered_df = filtered_df.filter(abs(zi_total_amount) <= 3.5)

    return filtered_df

In [0]:
# execute task 1.2
trips_12 = t12_remove_outliers(trips_11)

print_count(trips_12)
display(trips_12.take(10))

Row count: 2,462,734


PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
90,68,0.8,1.0,8.8
113,90,0.9,1.0,8.8
88,232,2.8,1.0,13.8
79,249,1.4,1.0,12.3
142,238,2.0,0.0,12.3
114,90,1.6,1.0,12.8
90,144,1.8,1.0,13.3
114,48,2.0,1.0,14.75
48,152,5.7,1.0,22.3
234,148,1.8,1.0,14.75


## Task 2 - Compute new columns

### Task 2.1 - Zone names

Obtain the **start** and **end** zone names of each trip by joining the `trips` and `zone_names` datasets (i.e. by using the `zone_names` dataset as lookup table).

**Note:** The columns containing the start and end zone ids of each trip are named `PULocationID` and `DOLocationID`, respectively.

### Task 2.2 - Unit profitability

Compute the column `unit_profitability = total_amount / trip_distance`.

For this task, I noticed there are 'Null' values in `zone_names` dataframe which causes a type error when using `display` method. Thus, after doing the join between `trips` dataframe and `zone_names`, I'm removing any 'Null' values. 

In [0]:
# Your solution implementation to task 2.1 goes HERE
def t21_join_zones(df, zones_df = zone_names):
    # input: output of task 1.2 and zone_names dataset
    join_1 = df.join(zones_df, df.PULocationID == zones_df.LocationID)
    new_df = join_1.withColumnRenamed('Zone', 'start_zone_name').drop('LocationID', 'Borough', 'service_zone')

    join_2 = new_df.join(zones_df, new_df.DOLocationID == zones_df.LocationID)
    new_df = join_2.withColumnRenamed('Zone', 'end_zone_name').drop('LocationID', 'Borough', 'service_zone')

    new_df = new_df.filter(new_df['start_zone_name'].isNotNull() & new_df['end_zone_name'].isNotNull())
    
    return new_df

In [0]:
# execute task 2.1
trips_21 = t21_join_zones(trips_12, zones_df = zone_names)

print_count(trips_21)
#display(trips_21.take(10))


Row count: 2,459,737


In [0]:
# Your solution implementation to task 2.2 goes HERE
from pyspark.sql.functions import round

def t22_calc_profit(df):
    # input: output of task 2.1
    df = df.withColumn('unit_profitability', round(df.total_amount/df.trip_distance, 4))
    return df

In [0]:
# execute task 2.2
trips_22 = t22_calc_profit(trips_21)

print_count(trips_22)
#display(trips_22.take(10))

Row count: 2,459,737


## Task 3: Rank zones by traffic, passenger volume and profitability

### 3.1 - Summarise interzonal travel

Build a graph data structure of zone-to-zone traffic, representing aggregated data about trips between any two zones. The graph will have one node for each zone and one edge connecting each pair of zones. In addition, edges contain aggregate information about all trips between those zones. 

For example, zones Z1 and Z2 are connected by *two* edges: edge Z1 --> Z2 carries aggregate data about all trips that originated in Z1 and ended in Z2, and edge Z2 --> Z1 carries aggregate data about all trips that originated in Z2 and ended in Z1.

The aggregate information of interzonal travel must include the following data:

- `average_unit_profit` - the average unit profitability (calculated as `mean(unit_profitability)`).
- `trips_count` -- the total number of recorded trips.
- `total_passengers` -- the total number of passenger across all trips (sum of `passenger_count`).

This graph can be represented as a new dataframe, with schema:

\[`PULocationID`, `DOLocationID`, `average_unit_profit`, `trips_count`, `total_passengers` \]

__hint__: the `groupby()` operator produces a `pyspark.sql.GroupedData` structure. You can then calculate multiple aggregations from this using `pyspark.sql.GroupedData.agg()`: 
- https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.groupby.html
- https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.GroupedData.agg.html

### Task 3.2 - Obtain top-10 zones

For each of the following measures, report the top-10 zones _using their plain names you dereferenced in the previous step, not the codes_. Note that this requires ranking the nodes in different orders. Specifically, you need to calculate the following further aggregations:

- the **total** number of trips originating from Z. This is simply the sum of `trips_count` over all outgoing edges for Z, i.e., edges of the form Z -> \*
- the **average** profitability of a zone. This is the average of all `average_unit_profit` over all *outgoing* edges from Z.
- The **total** passenger volume measured as the **sum** of `total_passengers` carried in trips that originate from Z

In [0]:
## Your solution to task 3.1 goes HERE
from pyspark.sql.functions import mean, sum, count

def t31_summarise_trips(df):
    # input: output of task 2.2
    new_df = df.groupBy('PULocationID', 'DOLocationID').agg(
        round(mean('unit_profitability'), 4).alias('average_unit_profit'),
        count('*').alias('trips_count'),
        sum('passenger_count').alias('total_passengers'))
    return new_df

In [0]:
# execute task 3.1
graph = t31_summarise_trips(trips_22)

print_count(graph)
display(graph.take(10))

Row count: 12,479


PULocationID,DOLocationID,average_unit_profit,trips_count,total_passengers
90,231,7.4116,1119,1572.0
87,33,6.8172,105,145.0
90,142,6.8737,907,1276.0
114,100,7.477,464,703.0
234,144,9.1157,1485,2184.0
148,262,4.804,270,407.0
233,100,11.6215,809,1159.0
246,249,8.8201,1150,1713.0
170,179,5.0913,76,99.0
48,232,5.3553,335,482.0


For the function `t32_summarise_zones_pairs`, I had to include the action `collect` at the end of my final result because when running the whole pipeline without Task 1.2, the execution time was not appropiate due to it was executing what it was in-memory. 

In [0]:
# Your solution to task 3.2 goes HERE (implement each of the functions below)
def t32_summarise_zones_pairs(df, zones_df = zone_names):
    # input: output of task 3.1 
    new_df = df.groupBy('PULocationID').agg(
        round(mean('average_unit_profit'), 4).alias('average_unit_profit'),
        sum('trips_count').alias('trips_count'),
        sum('total_passengers').alias('total_passengers'))
    join = new_df.join(zones_df, new_df.PULocationID == zones_df.LocationID)
    new_df = join.withColumnRenamed('Zone', 'start_zone_name').drop('LocationID', 'Borough', 'service_zone')
    new_df = new_df.collect()
    return spark.createDataFrame(new_df)

# Top 10 ranked zones by traffic (trip volume)
def t32_top10_trips(df_zones):
    # input: output of task 3.2
    df = df_zones.orderBy(df_zones['trips_count'].desc()).limit(10)
    return df

# Top 10 ranked zones by profit
def t32_top10_profit(df_zones):
    # input: output of task 3.2
    df = df_zones.orderBy(df_zones['average_unit_profit'].desc()).limit(10)
    return df

# Top 10 ranked zones by passenger volume
def t32_top10_passenger(df_zones):
    # input: output of task 3.2
    df = df_zones.orderBy(df_zones['total_passengers'].desc()).limit(10)
    return df

In [0]:
# execute task 3.2
zones = t32_summarise_zones_pairs(graph)

top10_trips     = t32_top10_trips(zones)
top10_profit    = t32_top10_profit(zones)
top10_passenger = t32_top10_passenger(zones)

In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
display(top10_trips)

PULocationID,average_unit_profit,trips_count,total_passengers,start_zone_name
237,6.5128,121668,169759.0,Upper East Side South
236,6.5801,103350,143461.0,Upper East Side North
186,7.5516,96129,136402.0,Penn Station/Madison Sq West
161,7.1772,93278,134523.0,Midtown Center
170,7.3896,88077,123468.0,Murray Hill
162,6.8585,84916,119494.0,Midtown East
142,6.9065,80607,114326.0,Lincoln Square East
239,6.4293,74326,105314.0,Upper West Side South
48,8.4293,74253,106856.0,Clinton East
141,6.1824,74210,103657.0,Lenox Hill West


In [0]:
# use 'display()' return a pandas DataFrame for 'pretty' output
display(top10_profit)

PULocationID,average_unit_profit,trips_count,total_passengers,start_zone_name
10,126.0682,277,101.0,Baisley Park
190,93.0251,72,63.0,Prospect Park
207,74.8283,27,27.0,Saint Michaels Cemetery/Woodside
241,49.3895,184,46.0,Van Cortlandt Village
59,40.281,6,3.0,Crotona Park
115,38.65,2,2.0,Grymes Hill/Clifton
117,35.8333,24,4.0,Hammels/Arverne
81,35.545,130,16.0,Eastchester
138,35.4384,9014,13575.0,LaGuardia Airport
89,33.8949,625,109.0,Flatbush/Ditmas Park


In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
display(top10_passenger)

PULocationID,average_unit_profit,trips_count,total_passengers,start_zone_name
237,6.5128,121668,169759.0,Upper East Side South
236,6.5801,103350,143461.0,Upper East Side North
186,7.5516,96129,136402.0,Penn Station/Madison Sq West
161,7.1772,93278,134523.0,Midtown Center
170,7.3896,88077,123468.0,Murray Hill
162,6.8585,84916,119494.0,Midtown East
142,6.9065,80607,114326.0,Lincoln Square East
48,8.4293,74253,106856.0,Clinton East
79,7.2534,73464,105381.0,East Village
239,6.4293,74326,105314.0,Upper West Side South


## Task 4 - Record the pipeline's execution time

Record the execution time of:

1. the whole pipeline
2. the whole pipeline except task 1.2

on the two tables below, for all dataset sizes: `'S'`, `'M'`, `'L'`, `'XL'`, `'XXL'`, and data formats: `parquet` and `delta`.

Analyse the resulting execution times and comment on the effect of dataset size, dataset format and task complexity (with and without task 1.2) on pipeline performance.

In [0]:
# CHANGE the value of the following arguments to record the pipeline execution times for increasing dataset sizes
SIZE = 'XXL'
DATA_FORMAT = 'delta'
WITH_TASK_12 = True

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)


    Trips dataset loaded!
    ---
      Size: XXL
      Format: delta
      Tables loaded: /FileStore/tables/taxi/delta/taxi-XXL-delta/
      Number of trips (dataset rows): 132,396,785
    


In [0]:
# run and record the resulting execution time shown by databricks (on the cell footer)

# IMPORTANT: this function calls all task functions in order of occurrence. For this code to run without errors, you have to load into memory all of the previous task-specific functions, even if you haven't implemented these yet.
pipeline(trips, with_task_12 = WITH_TASK_12)

[DataFrame[PULocationID: bigint, average_unit_profit: double, trips_count: bigint, total_passengers: double, start_zone_name: string],
 DataFrame[PULocationID: bigint, average_unit_profit: double, trips_count: bigint, total_passengers: double, start_zone_name: string],
 DataFrame[PULocationID: bigint, average_unit_profit: double, trips_count: bigint, total_passengers: double, start_zone_name: string]]

_Table 1. Pipeline performance for `parquet` format._

| metric                           | S     | M      | L      | XL     | XXL     |
|----------------------------------|-------|--------|--------|--------|---------|
| rows (M)                         | 2.898 | 15.571 | 41.953 | 90.443 | 132.396 |
| execution time (sec)  (w/o 1.2)  | 1.02  | 3.84   | 7.89   | 9.57   |  10.28  |
| execution time (sec)             | 2.32  |   8.58 | 23.54  | 46.59| 56.73 |
| sec / 1M records (w/o 1.2)       | 0.351   | 0.246    | 0.188    | 0.105    | 0.077     |
| sec / 1M records                 | 0.800   | 0.551   | 0.561    | 0.515    | 0.428     |

_Table 2. Pipeline performance for `delta` format._

| metric                      | S    | M    | L    | XL   | XXL    |
|-----------------------------|------|------|------|------|--------|
| rows (M)                    | 2.898|15.571|41.953|90.443|132.396 |
| execution time (sec)(w/o 1.2)  | 1.20 | 2.27 | 3.70 | 5.34 | 5.98   |
| execution time (sec)             | 2.25 | 3.01 | 7.09 |  9.39|  16.91 |
| sec / 1M records (w/o 1.2)  | 0.414  | 0.145  | 0.088  | 0.059  | 0.045    |
| sec / 1M records            | 0.776  | 0.193  | 0.168  | 0.103  | 0.1277    |

**Note:** I run the pipelines in CSC8641

In Table 1 and Table 2, we demonstrate that the execution time (with or without Task 1.2) increases gradually when the size of the dataset increases. However, the execution time per record is smaller when the size of the dataset is larger, showing the advantages of Spark for managing large datasets. 

Moreover, in both tables, we show that the execution time with Task 1.2 is larger. This happens because in this task we invoked two times actions like `collect()` and `selectExpr()` to calculate the median and MAD value for each column (`trip_distance` and `total_amount`). These operations are expensive potentially increasing the communication overhead because all the data needs to be collected to the driver node breaking the parallelism. 

Additionally, Parquet files are immutable, thus, every update to the file means that it rewrites the entire file. In contrast, Delta files can be updated without rewriting the whole file which reduces overhead and improves performance for operations like joins. Therefore, we can see that in Table 2, in which datasets are in Delta format, the execution times (with or without Task 1.2) are considerably smaller for all dataset sizes compared to the execution times in Table 1, in which datasets are in Parquet format.